<a href="https://colab.research.google.com/github/pinggu95/deep_dive_AI/blob/main/56%EC%9D%BC%EC%B0%A8/56%EC%9D%BC%EC%B0%A8_Latent_Dirichlet_Allocation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pandas==1.3.1

  Using cached pandas-1.3.1.tar.gz (4.7 MB)
  error: subprocess-exited-with-error
  
  × pip subprocess to install build dependencies did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Installing build dependencies ... error
error: subprocess-exited-with-error

× pip subprocess to install build dependencies did not run successfully.
│ exit code: 1
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.


In [ ]:
pip install pyLDAvis

In [ ]:
import pandas as pd
from sklearn.datasets import fetch_20newsgroups
import nltk
from nltk.corpus import stopwords
import gensim
from gensim import corpora

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
dataset = fetch_20newsgroups(shuffle=True, random_state=1, remove=('headers', 'footers', 'quotes'))
documents = dataset.data
print('샘플의 수 :',len(documents))

샘플의 수 : 11314


In [ ]:
documents[1]

"\n\n\n\n\n\n\nYeah, do you expect people to read the FAQ, etc. and actually accept hard\natheism?  No, you need a little leap of faith, Jimmy.  Your logic runs out\nof steam!\n\n\n\n\n\n\n\nJim,\n\nSorry I can't pity you, Jim.  And I'm sorry that you have these feelings of\ndenial about the faith you need to get by.  Oh well, just pretend that it will\nall end happily ever after anyway.  Maybe if you start a new newsgroup,\nalt.atheist.hard, you won't be bummin' so much?\n\n\n\n\n\n\nBye-Bye, Big Jim.  Don't forget your Flintstone's Chewables!  :) \n--\nBake Timmons, III"

In [ ]:
print(dataset.target_names)

['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


In [ ]:
news_df = pd.DataFrame({'document':documents})
# 특수 문자 제거
news_df['clean_doc'] = news_df['document'].str.replace("[^a-zA-Z]", " ")
# 길이가 3이하인 단어는 제거 (길이가 짧은 단어 제거)
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))
# 전체 단어에 대한 소문자 변환
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: x.lower())

In [ ]:
news_df['clean_doc'][1]

"yeah, expect people read faq, etc. actually accept hard atheism? need little leap faith, jimmy. your logic runs steam! jim, sorry can't pity you, jim. sorry that have these feelings denial about faith need well, just pretend that will happily ever after anyway. maybe start newsgroup, alt.atheist.hard, won't bummin' much? bye-bye, jim. don't forget your flintstone's chewables! bake timmons,"

In [ ]:
stop_words = stopwords.words('english')
tokenized_doc = news_df['clean_doc'].apply(lambda x: x.split())
tokenized_doc = tokenized_doc.apply(lambda x: [item for item in x if item not in stop_words])

In [ ]:
tokenized_doc[:5]

,clean_doc
0,"[well, sure, story, seem, biased., disagree, s..."
1,"[yeah,, expect, people, read, faq,, etc., actu..."
2,"[although, realize, principle, strongest, poin..."
3,"[notwithstanding, legitimate, fuss, proposal,,..."
4,"[well,, change, scoring, playoff, pool., unfor..."


In [ ]:
dictionary = corpora.Dictionary(tokenized_doc)
corpus = [dictionary.doc2bow(text) for text in tokenized_doc]

print(corpus[1])

[(59, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 1), (67, 1), (68, 1), (69, 1), (70, 1), (71, 1), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1), (78, 1), (79, 1), (80, 1), (81, 1), (82, 2), (83, 1), (84, 1), (85, 1), (86, 1), (87, 1), (88, 1), (89, 2), (90, 1), (91, 1), (92, 1), (93, 1), (94, 1), (95, 1), (96, 2), (97, 1), (98, 1), (99, 1), (100, 1), (101, 1), (102, 1)]


In [ ]:
print(dictionary[66])

bye-bye,


In [ ]:
len(dictionary)

181839

In [ ]:
NUM_TOPICS = 20
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)

topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

(0, '0.007*"radar" + 0.005*"detector" + 0.004*"cross" + 0.004*"chinese"')
(1, '0.011*"----------------------------------------------------------------------" + 0.011*"entry" + 0.007*"widget" + 0.006*"period"')
(2, '0.008*"would" + 0.007*"like" + 0.006*"good" + 0.005*"know"')
(3, '0.014*"jesus" + 0.009*"christian" + 0.008*"bible" + 0.007*"christians"')
(4, '0.009*"space" + 0.006*"national" + 0.005*"president" + 0.005*"april"')
(5, '0.017*"would" + 0.010*"people" + 0.008*"think" + 0.007*"like"')
(6, '0.006*"allocation" + 0.003*"christians." + 0.003*"countersteering" + 0.002*"motif."')
(7, '0.019*"armenian" + 0.015*"armenians" + 0.014*"turkish" + 0.009*"greek"')
(8, '0.013*"----" + 0.004*"financial" + 0.004*"5.25"" + 0.004*"period:"')
(9, '0.015*"55.0" + 0.003*"temp" + 0.003*"hong" + 0.003*"0.333"')
(10, '0.008*"cubs" + 0.006*"borland" + 0.003*"astros" + 0.003*"suck"')
(11, '0.009*"file" + 0.007*"using" + 0.007*"please" + 0.007*"also"')
(12, '0.018*"file:" + 0.003*"[kk]" + 0.002*"zealand"

In [ ]:
import pyLDAvis.gensim_models

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(ldamodel, corpus, dictionary)
pyLDAvis.display(vis)

In [ ]:
for i, topic_list in enumerate(ldamodel[corpus]):
  if i==5:
    break
  print(i,'번째 문서의 topic 비율은',topic_list)

0 번째 문서의 topic 비율은 [(2, 0.48656446), (4, 0.12776741), (5, 0.17263596), (13, 0.20086364)]
1 번째 문서의 topic 비율은 [(2, 0.1283348), (3, 0.14670303), (5, 0.43370855), (13, 0.06639828), (14, 0.20694867)]
2 번째 문서의 topic 비율은 [(2, 0.07490754), (5, 0.68063504), (8, 0.019273683), (13, 0.21160004)]
3 번째 문서의 topic 비율은 [(2, 0.27352998), (4, 0.07494976), (5, 0.34097332), (8, 0.012761316), (14, 0.20202984), (15, 0.026544033), (17, 0.05973328)]
4 번째 문서의 topic 비율은 [(2, 0.7768274), (5, 0.14478455), (13, 0.04680523)]


In [ ]:
def make_topictable_per_doc(ldamodel, corpus):
    topic_table = pd.DataFrame()

    for i, topic_list in enumerate(ldamodel[corpus]):
        doc = topic_list[0] if ldamodel.per_word_topics else topic_list
        doc = sorted(doc, key=lambda x: (x[1]), reverse=True)

        # 모든 문서에 대해서 각각 아래를 수행
        for j, (topic_num, prop_topic) in enumerate(doc):
            if j == 0:
                topic_table = topic_table.append(pd.Series([int(topic_num), round(prop_topic,4), topic_list]), ignore_index=True)

            else:
                break
    return(topic_table)